# Tree Models for Project 3

## Setup

### Import Libraries

In [1]:
import pandas as pd
import os

from sklearn.model_selection import train_test_split

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report

from operator import itemgetter

### Set File Locations

In [2]:
# note that some of the raw data files are very large
# these very large files are located in a gitignored directory.

# cleaned, merged data
merged_data_csv = "../00_Data/cleaned_data/cleaned_merged_data.csv"

# data with site predictions
export_data_csv = "../00_Data/cleaned_data/data_with_site_predictions.csv"

## Import Data

In [3]:
# Import census data
data_df = pd.read_csv(merged_data_csv)

data_df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220423 entries, 0 to 220422
Data columns (total 240 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   fips_block_group                  220423 non-null  int64  
 1   state                             220423 non-null  float64
 2   state_name                        220423 non-null  object 
 3   county                            220423 non-null  float64
 4   county_name                       220423 non-null  object 
 5   tract                             220423 non-null  float64
 6   block_group                       220423 non-null  float64
 7   flag                              220423 non-null  float64
 8   land_area                         220423 non-null  float64
 9   aian_land                         220423 non-null  float64
 10  urbanized_area_pop_cen_2010       220423 non-null  float64
 11  urban_cluster_pop_cen_2010        220423 non-null  

## Prep Data

In [4]:
# Prepare the target
target = data_df["has_superfund"]
target_names = ["negative", "positive"]

In [5]:
# Prepare the features
# Drop all the columns that came in from the site data. This prevents 'trailing indicators' from getting into the model.
# Also drop any column that shouldn't mathematically matter, such as FIPS, tract, etc.
exclusion_list = ['fips_block_group',
            'state',
            'state_name',
            'county',
            'county_name',
            'tract',
            'block_group',
            'has_superfund',
            'fips_full',
            'address',
            'city',
            'date_added',
            'federal_facility_ind',
            'federal_register_url',
            'geocode_source',
            'latitude',
            'longitude',
            'site_epa_id',
            'site_name',
            'site_narrative_url',
            'site_progress_url',
            'site_score',
            'site_text'
            ]

feature_df = data_df.copy()
feature_df.drop(feature_df[exclusion_list],axis=1,inplace=True)
feature_names = feature_df.columns

feature_df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220423 entries, 0 to 220422
Data columns (total 217 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   flag                              220423 non-null  float64
 1   land_area                         220423 non-null  float64
 2   aian_land                         220423 non-null  float64
 3   urbanized_area_pop_cen_2010       220423 non-null  float64
 4   urban_cluster_pop_cen_2010        220423 non-null  float64
 5   rural_pop_cen_2010                220423 non-null  float64
 6   tot_population_cen_2010           220423 non-null  float64
 7   tot_population_acs_09_13          220423 non-null  float64
 8   males_cen_2010                    220423 non-null  float64
 9   males_acs_09_13                   220423 non-null  float64
 10  females_cen_2010                  220423 non-null  float64
 11  females_acs_09_13                 220423 non-null  

### train/test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(feature_df, target, random_state=42)

## Build Models

### Tree Classifier

In [7]:
%%time
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

Wall time: 1min 9s


In [8]:
# use a confusion matrix to inspect the score
clf_predictions = clf.predict(X_test)
clf_report = classification_report(y_test, clf_predictions)
print(clf_report)

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99     54790
         1.0       0.05      0.09      0.07       316

    accuracy                           0.99     55106
   macro avg       0.52      0.54      0.53     55106
weighted avg       0.99      0.99      0.99     55106



In [9]:
# sorted(zip(clf.feature_importances_, feature_names), reverse=True)
# build a dictionary of features and their importance, and then sort.
clf_feature_importance = {feature_names[i]: clf.feature_importances_[i] for i in range(len(feature_names))}
{k: v for k, v in sorted(clf_feature_importance.items(), key=lambda item: item[1], reverse = True)}

{'pct_pop_5_17_acs_09_13': 0.015738796064859416,
 'med_house_value_tr_acs_09_13': 0.01429913688770417,
 'pct_no_ph_srvc_acs_09_13': 0.013977121455568042,
 'pct_nh_white_alone_cen_2010': 0.013558960185465417,
 'land_area': 0.013389664825440906,
 'pct_pop_65plus_cen_2010': 0.012775437027465615,
 'pct_pop_25_44_acs_09_13': 0.012555118156855349,
 'pct_pub_asst_inc_acs_09_13': 0.01202137032933412,
 'low_response_score': 0.011992211635943134,
 'pct_diff_hu_1yr_ago_acs_09_13': 0.011605523286862608,
 'med_hhd_inc_tr_acs_09_13': 0.010434486512166772,
 'pop_65plus_cen_2010': 0.010015655677676328,
 'pct_no_plumb_acs_09_13': 0.00997141466743938,
 'pct_census_mail_returns_cen_2010': 0.009962284000214063,
 'mail_return_rate_cen_2010': 0.009877626009543714,
 'pct_pop_45_64_acs_09_13': 0.009867887039986423,
 'pct_hispanic_acs_09_13': 0.009745367512624802,
 'tot_vacant_units_acs_09_13': 0.009329447725052387,
 'pct_nh_white_alone_acs_09_13': 0.009328231238653383,
 'pct_female_no_hb_cen_2010': 0.00903097

### Random Forest Classifier

In [10]:
%%time
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

Wall time: 8min 32s


In [11]:
rf_predictions = rf.predict(X_test)
rf_report = classification_report(y_test, rf_predictions)
print(rf_report)

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     54790
         1.0       1.00      0.08      0.15       316

    accuracy                           0.99     55106
   macro avg       1.00      0.54      0.57     55106
weighted avg       0.99      0.99      0.99     55106



In [12]:
# build a dictionary of features and their importance, and then sort.
rf_feature_importance = {feature_names[i]: rf.feature_importances_[i] for i in range(len(feature_names))}
{k: v for k, v in sorted(rf_feature_importance.items(), key=lambda item: item[1], reverse = True)}

{'land_area': 0.013223358411059899,
 'med_hhd_inc_tr_acs_09_13': 0.0077333198955994165,
 'pct_males_acs_09_13': 0.007608298353196141,
 'pct_females_cen_2010': 0.007549377380219747,
 'pct_nh_white_alone_cen_2010': 0.007499083387985542,
 'med_house_value_tr_acs_09_13': 0.007496475290740352,
 'pct_females_acs_09_13': 0.007425775497139278,
 'pct_males_cen_2010': 0.007370293480111875,
 'pct_nh_white_alone_acs_09_13': 0.0067542156522599735,
 'pct_diff_hu_1yr_ago_acs_09_13': 0.006739858818683004,
 'pct_pop_5_17_acs_09_13': 0.0067323047775192995,
 'pct_pop_45_64_acs_09_13': 0.006499350836879912,
 'pct_college_acs_09_13': 0.006239221771354093,
 'pct_census_uaa_cen_2010': 0.006130824308415036,
 'pct_prs_blw_pov_lev_acs_09_13': 0.0061212315711349635,
 'pct_census_mail_returns_cen_2010': 0.006108281692972736,
 'pct_hispanic_cen_2010': 0.006100654271387058,
 'pct_mailback_count_cen_2010': 0.006094453123217765,
 'pct_pop_45_64_cen_2010': 0.00609375005448901,
 'pct_one_health_ins_acs_09_13': 0.006081

### Balanced Tree Classifier

In [13]:
%%time
bclf = tree.DecisionTreeClassifier(class_weight='balanced')
bclf = bclf.fit(X_train, y_train)

Wall time: 41.4 s


In [14]:
bclf_predictions = bclf.predict(X_test)
bclf_report = classification_report(y_test, bclf_predictions)
print(bclf_report)

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99     54790
         1.0       0.06      0.09      0.07       316

    accuracy                           0.99     55106
   macro avg       0.53      0.54      0.53     55106
weighted avg       0.99      0.99      0.99     55106



In [15]:
# build a dictionary of features and their importance, and then sort.
bclf_feature_importance = {feature_names[i]: bclf.feature_importances_[i] for i in range(len(feature_names))}
# {k: v for k, v in sorted(bclf_feature_importance.items(), key=lambda item: item[1], reverse = True)}

### Balanced Random Forest Classifier

In [16]:
%%time
brf = RandomForestClassifier(n_estimators=200, class_weight='balanced')
brf = brf.fit(X_train, y_train)

Wall time: 5min 47s


In [17]:
brf_predictions = brf.predict(X_test)
brf_report = classification_report(y_test, brf_predictions)
print(brf_report)

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     54790
         1.0       1.00      0.08      0.15       316

    accuracy                           0.99     55106
   macro avg       1.00      0.54      0.57     55106
weighted avg       0.99      0.99      0.99     55106



In [18]:
# build a dictionary of features and their importance, and then sort.
brf_feature_importance = {feature_names[i]: brf.feature_importances_[i] for i in range(len(feature_names))}
{k: v for k, v in sorted(brf_feature_importance.items(), key=lambda item: item[1], reverse = True)}

{'land_area': 0.05422911086873596,
 'pct_females_cen_2010': 0.010183164243196126,
 'pct_males_cen_2010': 0.010011283378453029,
 'pct_single_unit_acs_09_13': 0.00947204942698851,
 'pct_females_acs_09_13': 0.00825721743950478,
 'pct_college_acs_09_13': 0.008154621277228359,
 'pct_males_acs_09_13': 0.007913726765431676,
 'pct_pop_65plus_cen_2010': 0.007890656890217387,
 'pct_nh_white_alone_cen_2010': 0.007774720655569643,
 'pct_mailback_count_cen_2010': 0.007399209812303214,
 'pct_mlt_u2_9_strc_acs_09_13': 0.007205503785510215,
 'pct_pop_65plus_acs_09_13': 0.0071899652535765204,
 'mail_return_rate_cen_2010': 0.00712651414993236,
 'pct_urbanized_area_pop_cen_2010': 0.007000895054477424,
 'pct_rural_pop_cen_2010': 0.00697813608678997,
 'med_house_value_tr_acs_09_13': 0.006939583768664142,
 'rural_pop_cen_2010': 0.006925415222432565,
 'pct_census_uaa_cen_2010': 0.006846035346726375,
 'pct_pop_18_24_acs_09_13': 0.006840772480555399,
 'pct_one_health_ins_acs_09_13': 0.006809581909859804,
 'pct

## Evaluate Models

### Confusion Matrices

In [19]:
print('Confusion Matrices')
print('-----------------------------------------------------')
print('Decision Tree')
print(clf_report)
print('-----------------------------------------------------')
print('Random Forest')
print(rf_report)
print('-----------------------------------------------------')
print('Decision Tree with Class Balancing')
print(bclf_report)
print('-----------------------------------------------------')
print('Random Forest with Class Balancing')
print(brf_report)
print('-----------------------------------------------------')

Confusion Matrices
-----------------------------------------------------
Decision Tree
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99     54790
         1.0       0.05      0.09      0.07       316

    accuracy                           0.99     55106
   macro avg       0.52      0.54      0.53     55106
weighted avg       0.99      0.99      0.99     55106

-----------------------------------------------------
Random Forest
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     54790
         1.0       1.00      0.08      0.15       316

    accuracy                           0.99     55106
   macro avg       1.00      0.54      0.57     55106
weighted avg       0.99      0.99      0.99     55106

-----------------------------------------------------
Decision Tree with Class Balancing
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99    

### ROC-AUC Score

In [20]:
from sklearn.metrics import roc_auc_score

clf_prob_predictions = clf.predict_proba(X_test)
bclf_prob_predictions = bclf.predict_proba(X_test)
rf_prob_predictions = rf.predict_proba(X_test)
brf_prob_predictions = brf.predict_proba(X_test)

In [21]:
print(f"Decision Tree ROC-AUC score: {roc_auc_score(y_test, clf_prob_predictions[:,1])}")
print(f"Balanced Decision Tree ROC-AUC score: {roc_auc_score(y_test, bclf_prob_predictions[:,1])}")
print(f"Random Forest ROC-AUC score: {roc_auc_score(y_test, rf_prob_predictions[:,1])}")
print(f"Balanced Random Forest ROC-AUC score: {roc_auc_score(y_test, brf_prob_predictions[:,1])}")

Decision Tree ROC-AUC score: 0.5412866965005626
Balanced Decision Tree ROC-AUC score: 0.5403432207207728
Random Forest ROC-AUC score: 0.6368371700000693
Balanced Random Forest ROC-AUC score: 0.6752552034118765


## Reduce Features and Iterate

In [22]:
# Select the bottom K features for exclusion
K = 187
bottom_features = dict(sorted(brf_feature_importance.items(), key = itemgetter(1))[:K])
# make a list of the feature names
bottom_list = list(bottom_features.keys())

In [23]:
# make a new exclusion list and generate a new features df
new_exclusion_list = exclusion_list + bottom_list

new_feature_df = data_df.copy()
new_feature_df.drop(new_feature_df[new_exclusion_list],axis=1,inplace=True)
feature_names = new_feature_df.columns

new_feature_df.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220423 entries, 0 to 220422
Data columns (total 30 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   land_area                        220423 non-null  float64
 1   rural_pop_cen_2010               220423 non-null  float64
 2   mlt_u2_9_strc_acs_09_13          220423 non-null  float64
 3   med_house_value_bg_acs_09_13     220423 non-null  float64
 4   med_house_value_tr_acs_09_13     220423 non-null  float64
 5   mail_return_rate_cen_2010        220423 non-null  float64
 6   low_response_score               220423 non-null  float64
 7   pct_urbanized_area_pop_cen_2010  220423 non-null  float64
 8   pct_rural_pop_cen_2010           220423 non-null  float64
 9   pct_males_cen_2010               220423 non-null  float64
 10  pct_males_acs_09_13              220423 non-null  float64
 11  pct_females_cen_2010             220423 non-null  float64
 12  pc

In [24]:
%%time
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(new_feature_df, target, random_state=42)

brf_2 = RandomForestClassifier(n_estimators=200, class_weight='balanced')
brf_2 = brf_2.fit(X_train_2, y_train_2)

brf_2_predictions = brf_2.predict(X_test_2)
brf_2_prob_predictions = brf_2.predict_proba(X_test_2)


print(classification_report(y_test_2, brf_2_predictions))

print(f"ROC-AUC score: {roc_auc_score(y_test_2, brf_2_prob_predictions[:,1])}")

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     54790
         1.0       1.00      0.08      0.15       316

    accuracy                           0.99     55106
   macro avg       1.00      0.54      0.57     55106
weighted avg       0.99      0.99      0.99     55106

ROC-AUC score: 0.6594301082845664
Wall time: 2min 49s


In [25]:
# %%time
# best_features = 0
# best_roc_auc_score = 0
# score_log = {}

# for i in range(0, len(feature_df.columns)-1):
    
#     print('-----------------------------------------------------')
#     print(f"Top {len(feature_df.columns)-i} features")
    
#     # trim off a bunch of features
#     K = i
#     bottom_features = dict(sorted(brf_feature_importance.items(), key = itemgetter(1))[:K])
    
#     # make a list of the feature names
#     bottom_list = list(bottom_features.keys())
#     new_exclusion_list = exclusion_list + bottom_list

#     # create the new feature list
#     new_feature_df = data_df.copy()
#     new_feature_df.drop(new_feature_df[new_exclusion_list],axis=1,inplace=True)
#     feature_names = new_feature_df.columns

#     # create the train/test split
#     X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(new_feature_df, target, random_state=42)

#     # create and fit the model
#     brf_2 = RandomForestClassifier(n_estimators=200, class_weight='balanced')
#     brf_2 = brf_2.fit(X_train_2, y_train_2)

#     # run some predictions
#     brf_2_predictions = brf_2.predict(X_test_2)
#     brf_2_prob_predictions = brf_2.predict_proba(X_test_2)
    
#     new_score = roc_auc_score(y_test_2, brf_2_prob_predictions[:,1])
    
#     score_log[len(feature_df.columns)-i] = new_score
    
#     if new_score > best_roc_auc_score:
#         best_roc_auc_score = new_score
#         best_features = len(feature_df.columns)-i
        
#     # print results
#     print(classification_report(y_test_2, brf_2_predictions))
#     print(f"ROC-AUC score: {new_score}")
#     print('-----------------------------------------------------')

In [26]:
# print(f"Best results was a ROC-AUC score of {best_roc_auc_score} for the top {best_features} features.")

In [27]:
# print(score_log)

# Finish chosen model

In [32]:
# select the chosen model
model = brf_2

## Save Model

In [33]:
import joblib
filename = 'superfund_site_model.sav'
joblib.dump(model, filename)

['superfund_site_model.sav']

## Generate Predictions

In [34]:
# get features for generating predictions for full dataset
new_feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220423 entries, 0 to 220422
Data columns (total 30 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   land_area                        220423 non-null  float64
 1   rural_pop_cen_2010               220423 non-null  float64
 2   mlt_u2_9_strc_acs_09_13          220423 non-null  float64
 3   med_house_value_bg_acs_09_13     220423 non-null  float64
 4   med_house_value_tr_acs_09_13     220423 non-null  float64
 5   mail_return_rate_cen_2010        220423 non-null  float64
 6   low_response_score               220423 non-null  float64
 7   pct_urbanized_area_pop_cen_2010  220423 non-null  float64
 8   pct_rural_pop_cen_2010           220423 non-null  float64
 9   pct_males_cen_2010               220423 non-null  float64
 10  pct_males_acs_09_13              220423 non-null  float64
 11  pct_females_cen_2010             220423 non-null  float64
 12  pc

In [35]:
# generate predictions using the final selected model
site_presence_prediction = model.predict_proba(new_feature_df)

In [36]:
# make a list of the predictions
predicted_probabilities = []
for i in range(0, len(site_presence_prediction)):
    predicted_probabilities.append(site_presence_prediction[i][1])

In [37]:
# add predictions to the full dataset for export
export_df = data_df.copy()
export_df['site_probability'] = predicted_probabilities
export_df['score_prediction'] = 0.0

In [38]:
export_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220423 entries, 0 to 220422
Columns: 242 entries, fips_block_group to score_prediction
dtypes: float64(228), int64(1), object(13)
memory usage: 407.0+ MB


## Export to csv

In [39]:
export_df.to_csv(export_data_csv, index = False)

## Export feature list

In [48]:
tree_features = new_feature_df.columns.values.tolist()
tree_features

['land_area',
 'rural_pop_cen_2010',
 'mlt_u2_9_strc_acs_09_13',
 'med_house_value_bg_acs_09_13',
 'med_house_value_tr_acs_09_13',
 'mail_return_rate_cen_2010',
 'low_response_score',
 'pct_urbanized_area_pop_cen_2010',
 'pct_rural_pop_cen_2010',
 'pct_males_cen_2010',
 'pct_males_acs_09_13',
 'pct_females_cen_2010',
 'pct_females_acs_09_13',
 'pct_pop_18_24_cen_2010',
 'pct_pop_18_24_acs_09_13',
 'pct_pop_25_44_cen_2010',
 'pct_pop_45_64_cen_2010',
 'pct_pop_65plus_cen_2010',
 'pct_pop_65plus_acs_09_13',
 'pct_hispanic_cen_2010',
 'pct_nh_white_alone_cen_2010',
 'pct_not_hs_grad_acs_09_13',
 'pct_college_acs_09_13',
 'pct_one_health_ins_acs_09_13',
 'pct_sngl_prns_hhd_acs_09_13',
 'pct_owner_occp_hu_cen_2010',
 'pct_single_unit_acs_09_13',
 'pct_mlt_u2_9_strc_acs_09_13',
 'pct_census_uaa_cen_2010',
 'pct_mailback_count_cen_2010']

In [55]:
site_model_features = new_feature_df.columns.values.tolist()
f = open("feature_list_vault.py", "a")
hasTextBeenWritten = False
f.write("site_model_features = [ \n")
for i in site_model_features: 
    if hasTextBeenWritten:
        f.write(",\n")
    f.write(f"'{i}'")
    hasTextBeenWritten = True
f.write("\n]\n\n")
f.close()